In [5]:
import numpy as np
import os
import json
import sys
import torch

In [26]:
def read_input(file_path):
    with open(file_path) as json_data:
        d = json.load(json_data)

    return d['positions'], d['rotations'], d['parents'], d['foot_contact']

def main():
    # reading all data
    file_path = "C:\\Users\\eggyr\\OneDrive\\RPI\\S10\\Projects in ML\\final\\lafan1_detail_model_benchmark_5_0-2231.json"
    all_orig_positions, all_orig_rotations, parents, all_foot_contact = read_input(file_path)
    all_orig_positions = np.array(all_orig_positions)
    all_orig_rotations = np.array(all_orig_rotations)
    new_joints = 10
    all_new_positions = np.array((all_orig_positions.shape[0], all_orig_positions.shape[1], new_joints, 3))
    all_new_rotations = np.array((all_orig_rotations.shape[0], all_orig_rotations.shape[1], new_joints, 3))
    print(f'original positions shape: {all_orig_positions.shape}')

    all_new_positions, all_new_rotations = conv_rig(all_orig_positions, all_orig_rotations, parents)
    #print(f"all pos: {all_new_positions}\nall rot: {all_new_rotations}")
    return
    for sequence in range(all_orig_positions.shape[0]):
        all_new_positions[sequence], all_new_rotations[sequence] = conv_rig(all_orig_positions[sequence], all_orig_rotations[sequence], parents)
        print(f"all pos: {all_new_positions}\nall rot: {all_new_rotations}")
        return


def conv_rig(positions, rotations, parents):
    # takes in original positions and rotations and returns the new representation of positions and rotations
    global_rot, global_pos = fk_torch(torch.from_numpy(rotations), torch.from_numpy(positions), parents)
    global_pos = global_pos.numpy()
    global_rot = global_rot.numpy()
    print(f"all pos {global_pos.shape}: {global_pos}\nall rot {global_rot.shape}: {global_rot}")
    return global_pos, global_rot

    
def fk_torch(lrot, lpos, parents):
    """
    Calculate forward kinematics.

    Args:
        lrot (Tensor): Local rotation of joints. Shape: (..., joints, 3, 3)
        lpos (Tensor): Local position of joints. Shape: (..., joints, 3)
        parents (list of int or 1D int Tensor): Parent indices.

    Returns:
        Tensor, Tensor: (global rotation, global position).
            Shape: (..., joints, 3, 3), (..., joints, 3)
    """
    gr = [lrot[..., :1, :, :]]
    gp = [lpos[..., :1, :]]

    print(f'{gr[0].shape}, {gp[0].shape}')

    for i in range(1, len(parents)):
        gr_parent = gr[parents[i]]
        gp_parent = gp[parents[i]]

        gr_i = torch.matmul(gr_parent, lrot[..., i:i + 1, :, :])
        gp_i = gp_parent + \
            torch.matmul(gr_parent, lpos[..., i:i + 1, :, None]).squeeze(-1)

        gr.append(gr_i)
        gp.append(gp_i)

    return torch.cat(gr, dim=-3), torch.cat(gp, dim=-2)

In [27]:
main()

original positions shape: (2232, 17, 22, 3)


AttributeError: 'list' object has no attribute 'shape'